In [1]:
import avro.schema
from avro.io import DatumWriter, DatumReader
from avro.datafile import DataFileWriter, DataFileReader
import tempfile
import numpy as np
from main import LZW
import struct

In [2]:
def pack(data):
    buffer = bytearray([])
    for el in data:
        buffer.extend(struct.pack("H", el+1))
    return buffer

def unpack(buffer: bytearray):
    i=0
    data = []
    while i < len(buffer):
        data.append(struct.unpack("H", buffer[i: i+2])[0]-1)
        i+=2
    return data

In [3]:
lzw = LZW()

with open("test.txt") as file:
    text = file.read()

In [4]:
enc = lzw.encode(text)
bin_data = pack(enc)

with open('test.bn', "wb") as file:
    file.write(bin_data)

In [5]:
unbin_data = unpack(bin_data)

In [6]:
lzw = LZW()
enc = lzw.encode(text)
schema = avro.schema.parse(open("schema.avsc", "rb").read())
data = {"sonnets": enc}
file = 'compressed_text.avro'

writer = DataFileWriter(open(file, "wb"), DatumWriter(), schema)
writer.append(data)
writer.close()

In [7]:
reader = DataFileReader(open(file, "rb"), DatumReader())
for t in reader:
    test = t['sonnets']
    dec = lzw.decode(test)
    print(dec)
    
reader.close()

FROM fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou, contracted to thine own bright eyes,
Feed'st thy light'st flame with self-substantial fuel,
Making a famine where abundance lies,
Thyself thy foe, to thy sweet self too cruel.
Thou that art now the world's fresh ornament
And only herald to the gaudy spring,
Within thine own bud buriest thy content
And, tender churl, makest waste in niggarding.
Pity the world, or else this glutton be,
To eat the world's due, by the grave and thee.
When forty winters shall beseige thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery, so gazed on now,
Will be a tatter'd weed, of small worth held:
Then being ask'd where all thy beauty lies,
Where all the treasure of thy lusty days,
To say, within thine own deep-sunken eyes,
Were an all-eating shame and thriftless praise.
How much more praise deserved thy b

In [8]:
!pip install lzw3


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!python -m lzw3.compressor -k -t -v -d test.txt

[16:19:05] {MAIN} Executing LZW task with following options:
	recursive:  False
	verbose:    True
	time:       True
	keep:       True
	force:      False
	debug:      True
	files:      ['test.txt']
[16:19:05] {COMPRESSOR} Initializing sequence table; alphabet size = 256
[16:19:05] {COMPRESSOR} Compressing file 'test.txt' to 'test.txt.Z'
'test.txt' compressed from 18.5K to 9.1K - space saved = 50.7% (12ms)
